# Max Hand Wins
The following code is the project work for the course ["Python OOP Course: Master Object-Oriented Programming"](https://www.udemy.com/course/python-oop-course/) by Dr. Karim Elghamrawy on Udemy. I have used Python and OOP to simulate a *maximum-hand-wins* game based on playing cards. The rules of the game are as follows.

# Rules

In this game, three players pick two cards each from a shuffled deck of playing cards per round. Each player then presents the stronger of the two cards they've picked. The player having the strongest among the presented cards is the winner for that round and gets a point. The player with the highest number of points at the end of the game is the winner. For two given cards, the one having a higher 'rank' (2 < 3 < ... < 9 < Jack < Queen < King < Ace) is stronger. In case the ranks are identical, the card having a higher 'suit' (clubs < diamonds < hearts < spades) is stronger among the two. 

The game, according to my implementation, can have --
1. An arbitrary number of players can be present and
2. Each player can pick any integer number of cards
3. The Game can either last for a given number of rounds or until the cards in the deck are exhausted

# Code:

## Load Python Libraries
Load a few built-in Python libraries.

In [ ]:
from enum import Enum 
from functools import total_ordering
import random
import time

# uncomment for python scripts
#import os

# uncomment for jupyter lab/notebook
from IPython.display import clear_output

## Class definitions
1. `Rank`: Integer mapping for playing card ranks (sub-class of `Enum`)
2. `Suit`: Integer mapping for playing card suits (sub-class of `Enum`)
3. `PlayingCard`: Playing card attributes and comparison methods
4. `Deck`: Number of decks, shuffling, picking cards, etc.
5. `Game`: Play game method, keep track of score, etc.   

In [ ]:
class Rank(Enum):
    '''
    Subclass of Enum to map integers to the ranks of playing cards
    '''
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    JACK = 11
    QUEEN = 12
    KING = 13
    ACE = 14

class Suit(Enum):
    '''
    Subclass of Enum to map integers to the suits of playing cards
    '''
    SPADES = 0
    HEARTS = 1
    DIAMONDS = 2
    CLUBS = 3

# total_ordering decorator so that not all comparison
# operators need to be defined explicitly
@total_ordering
class PlayingCard:
    # constructor
    def __init__(self, rank, suit):
        self._rank = rank
        self._suit = suit

    # string representation of the object
    def __str__(self):
        return f'{self._rank.name}/{self._suit.name}'
    
    def get_rank(self):
        return self._rank

    def get_suit(self):
        return self._suit

    def prettyprint_card(self):
        return f'||{self._rank.name.center(8)}/{self._suit.name.center(10)}||'

    # Comparison operations for two Playing card using dunder methods
    # Only 'equality' and 'greater than' operations are defined,
    # others are deduced by python and total_ordering decorator.
    def __eq__(self,other):
        return (self._rank.value == other._rank.value) and (self._suit.value == other._suit.value) 

    def __gt__(self,other):
        if (self == other): return False
        if (self._rank.value > other._rank.value):
            return True
        elif (self._rank.value < other._rank.value):
            return False
        else:
            if (self._suit.value > other._suit.value):
                return True
            elif (self._suit.value < other._suit.value):
                return False

class Deck:
    _ranks = [Rank.TWO, Rank.THREE, Rank.FOUR,
              Rank.FIVE, Rank.SIX, Rank.SEVEN,
              Rank.EIGHT, Rank.NINE, Rank.TEN,
              Rank.JACK, Rank.QUEEN, Rank.KING,
              Rank.ACE]

    _suits = [Suit.SPADES, Suit.HEARTS,
              Suit.DIAMONDS, Suit.CLUBS]

    _a_set_of_playing_cards = []
    for suit in _suits:
        for rank in _ranks:
            _a_set_of_playing_cards.append(PlayingCard(rank,suit))

    def __init__(self,number_of_bundles=1):
        self._deck = []
        self._number_of_bundles = number_of_bundles
        for _ in range(self._number_of_bundles):
            self._deck.extend(self._a_set_of_playing_cards)

    def get_deck(self):
        return self._deck

    def shuffle_deck(self,times=10):
        for _ in range(times):
            random.shuffle(self._deck)

    def draw(self,cards_per_draw=2):
        if (len(self._deck) < cards_per_draw):
            print (f'There are fewer than {cards_per_draw} cards left in the deck!')
            return None
        draw_list = []
        for i in range(cards_per_draw):
           draw_list.append(self._deck.pop())
        draw_list.reverse()
        return draw_list

    def get_no_cards_in_deck(self):
        return len(self._deck)

class Game:
    '''
    An instantiation of the Game class contains the list of
    players, score card, cards draw, and strongest hand.
    '''
    def __init__(self,players,deck,cards_per_draw=2,number_of_rounds=100):
        self._players = players # list of players. use the Player class
        self._deck = deck       # Supply the shuffled deck
        self._number_of_rounds = number_of_rounds       # Supply the shuffled deck
        self._cards_per_draw = cards_per_draw
        
        self._number_of_players = len(self._players)
        self._score = {player: 0 for player in self._players}
        self._players_hands = {player: [] for player in self._players}
        self._players_strongest_hand = {player: [] for player in self._players}

    def show_score(self):
        return self._score

    def get_no_cards_in_deck(self):
        return self._deck.get_no_cards_in_deck()

    def play_round(self):
        for player in self._players:
            draw = self._deck.draw(self._cards_per_draw)
            if draw is None: return None
            self._players_hands[player].insert(0,draw)
            draw.sort()
            self._players_strongest_hand[player] = draw[-1]
            print(f'{player}:',end='\t')
            for card in draw:
                print(f'\t {card.prettyprint_card()}   ',end='')
                #print(f'\t {card}   ',end='')
            print('')
        strongest_hand = self._players[0]
        for player in self._players:
            if (self._players_strongest_hand[strongest_hand] < self._players_strongest_hand[player]):
                strongest_hand = player
        self._score[strongest_hand] += 1
        print ()
        print ('_'*100)
        print("Score card:", self._score)
        print ('_'*100)
        print ('')

    def play(self):
        for round in range(self._number_of_rounds):
            if( (self.get_no_cards_in_deck() < self._cards_per_draw * self._number_of_players)
                    or round == self._number_of_rounds):
                print('Ran out of cards!'.center(100))
                print('*** GAME OVER! ***'.center(100))
                print('')
                break
            #os.system('cls||clear') # uncomment for python scripts
            clear_output(wait=True) # uncomment for python scripts
            print ('_'*100)
            print (f'Round Number: {round+1}')
            print ('')
            game.play_round()
            time.sleep(0.3)
        print("Final Score card:", self._score)

## Play the game

In [ ]:
#if __name__ == "__main__": # uncomment for python scripts

deck = Deck(number_of_bundles = 10)
deck.shuffle_deck(times = 100)
players = ['Adam', 'Bob', 'Claire','David']

game = Game(players, deck, cards_per_draw=2, number_of_rounds = 50)
game.play()